In [1]:
import pandas as pd
from scipy import sparse
from tqdm import tqdm
from preprocessing.duration_matrix import DurationSparseMatrix
from scipy.spatial.distance import cosine

In [2]:
CONST_PATH = '../data/postprocessing/'

In [3]:
users_genres = pd.read_csv(f'{CONST_PATH}users_genres.csv', index_col='user_uid')
users_genres.head()

,игровой,шоу,музыка,комедия,мелодрама,драма,комедийный,фэнтезийный,боевик,приключение,...,молодёжь,детский,ситком,индонезия,психологический,увлечение,досуг,концерт,скетчкома,интервью
user_uid,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,400,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,27683,12981,12324,5154,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25,0,0,0,900,6920,10666,1559,0,14166,0,...,0,0,0,0,0,0,0,0,0,0
43,0,0,0,599,61,61,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45,0,0,0,1470,1938,4257,293,0,6029,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
users_types = pd.read_csv(f'{CONST_PATH}user_content_types.csv', index_col=0)
users_types.head()

,serial_with_season,movie,serial_without_season
1,0,6720,0
4,30169,23638,0
25,665,23969,0
43,0,660,0
45,0,14370,0


In [5]:
USER_ID = 4
genres_similar_users = users_genres.drop(USER_ID, axis=0).apply(
    lambda row: cosine(users_genres.loc[USER_ID], row), axis=1)

In [6]:
genres_similar_users = genres_similar_users.sort_values().head(500)

In [7]:
most_similar_index = genres_similar_users.index
most_similar_users_types = users_types.loc[most_similar_index]

In [8]:
most_similar_users_types.reset_index().iloc[:, 1:]

,serial_with_season,movie,serial_without_season
0,23797,24036,0
1,25555,11731,0
2,1845,590,0
3,26173,66217,0
4,33617,584,0
...,...,...,...
495,0,12253,0
496,0,36819,0
497,0,8350,0
498,0,15571,0


In [9]:
type_similar_users = most_similar_users_types.apply(
    lambda row: cosine(most_similar_users_types.reset_index().iloc[USER_ID, 1:], row), axis=1)

In [10]:
type_similar_users = type_similar_users.sort_values(kind='mergesort').head(100)

In [11]:
dsm = DurationSparseMatrix()
history, content = dsm.read_info()

users_films = dsm.get_matrix_part(type_similar_users.index)
user_info = dsm.get_matrix_part([USER_ID])

In [28]:
ser = users_films.sum()
films_to_drop = list(ser[ser < 0.5].index)
user_info.drop(columns=films_to_drop, inplace=True)
users_films.drop(columns=films_to_drop, inplace=True)

In [29]:
target_list = []
for column in user_info.columns:
    if user_info[column].values[0] == 0:
        target_list.append(column)

In [30]:
from catboost import CatBoostRegressor, CatBoostError

for target in tqdm(target_list):
    X = users_films.drop(columns=[target])
    Y = users_films[target]
    if Y.sum() != 0:
        model = CatBoostRegressor(verbose=0)
        model.fit(X, Y)
        user_info[target] = model.predict(user_info.drop(columns=[target]))
    else:
        user_info[target] = 0
        print('zero')

100%|██████████| 25/25 [00:26<00:00,  1.06s/it]


In [34]:
user_info[target_list].T.sort_values(by=USER_ID, ascending=False)

,4
25804,0.003157
19563,0.002793
21306,0.000707
26474,0.000682
16589,0.000565
13565,0.000410
20280,0.000364
21757,0.000310
17730,0.000275
12154,0.000248
